In [140]:
# Importando bibliotecas de EDA
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Importando bibliotecas para criar o modelo
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error, mean_squared_error

### Carregar Dados

In [141]:
# Carregar dados
df_costs = pd.read_csv('.\datasets\healthcosts.csv')

<>:2: SyntaxWarning:

invalid escape sequence '\d'

<>:2: SyntaxWarning:

invalid escape sequence '\d'

C:\Users\Carolina\AppData\Local\Temp\ipykernel_16640\900826670.py:2: SyntaxWarning:

invalid escape sequence '\d'



In [142]:
df_costs.head(10)

,age,sex,bmi,children,smoker,region,medical charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
5,31,female,25.740,0,no,southeast,3756.62160
6,46,female,33.440,1,no,southeast,8240.58960
7,37,female,27.740,3,no,northwest,7281.50560
8,37,male,29.830,2,no,northeast,6406.41070
9,60,female,25.840,0,no,northwest,28923.13692


In [143]:
df_costs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   age              1338 non-null   int64  
 1   sex              1338 non-null   object 
 2   bmi              1338 non-null   float64
 3   children         1338 non-null   int64  
 4   smoker           1338 non-null   object 
 5   region           1338 non-null   object 
 6   medical charges  1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


### Feature Engineering

In [144]:
# Mostrar e Remover colunas categoricas que possuem somente um valor possível
for column in df_costs.select_dtypes(include=['object']).columns:
    if df_costs[column].nunique() == 1:
        print(f'Coluna {column} possui somente um valor possível: {df_costs[column].unique()}')

In [ ]:
# Mostrar os valores possíveis para todas as colunas categoricas
for column in df_costs.select_dtypes(include=['object']).columns:
    print(f'Coluna {column} possui esses valores possíveis: {df_costs[column].unique()}')

Coluna sex possui somente um valor possível: ['female' 'male']
Coluna smoker possui somente um valor possível: ['yes' 'no']
Coluna region possui somente um valor possível: ['southwest' 'southeast' 'northwest' 'northeast']


In [146]:
# Mostrar o percentual de valores ausentes para as colunas categoricas
for column in df_costs.select_dtypes(include=['object']).columns:
    contagem_nulas = df_costs[column].isnull().sum()
    print(f'{column}: {contagem_nulas / len(df_costs) * 100 :.2f}%')

sex: 0.00%
smoker: 0.00%
region: 0.00%


In [147]:
# Mostrar e Remover as colunas numéricas que possuem somente um valor possível
for column in df_costs.select_dtypes(include=['number']).columns:
    if df_costs[column].nunique() == 1:
        print(f'Coluna {column} possui somente um valor possível: {df_costs[column].unique()}')

In [148]:
# Mostrar o percentual de valores ausentes para as colunas numéricas
for column in df_costs.select_dtypes(include=['number']).columns:
    contagem_nulas = df_costs[column].isnull().sum()
    print(f'{column}: {contagem_nulas / len(df_costs) * 100 :.2f}%')

age: 0.00%
bmi: 0.00%
children: 0.00%
medical charges: 0.00%


In [149]:
# Converter colunas categoricas com valores Yes ou No para 1 e 0
for column in df_costs.select_dtypes(include=['object']).columns:
    valores_unicos = df_costs[column].unique()
    if set(valores_unicos).issubset(set(['yes', 'no'])):
        df_costs[column] == df_costs[column].apply(lambda x: 1 if x == 'yes' else 0)
        print(f'{column}: {df_costs[column].unique()}')

smoker: ['yes' 'no']


In [150]:
df_costs['smoker'] = df_costs['smoker'].apply(lambda x: 1 if x == 'yes' else 0)
# Foi colocado o peso de 1 e 0 pois é importante distinguir os resultados da variavel

### EDA

In [151]:
# Apresentar Estatística Descritiva
df_costs.describe()

,age,bmi,children,smoker,medical charges
count,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,0.204783,13270.422265
std,14.049960,6.098187,1.205493,0.403694,12110.011237
min,18.000000,15.960000,0.000000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,0.000000,9382.033000
75%,51.000000,34.693750,2.000000,0.000000,16639.912515
max,64.000000,53.130000,5.000000,1.000000,63770.428010


In [152]:
# Analisar a Variavel Target
# Mostrar distribuição de custos médicos
fig = px.histogram(df_costs, x='medical charges', nbins=30, title='Distribuição de Custo Médico')

fig.show()

In [153]:
# Mostrar distribuição de idade
fig = px.histogram(df_costs, x='age', nbins=30, title='Distribuição da Idade')

fig.show()

In [154]:
# Mostrar distribuição da filhos
fig = px.histogram(df_costs, x='children', title='Distribuição de Quantidade de Filhos')
fig.show()

In [155]:
# Mostrar distribuição de BMI
fig = px.histogram(df_costs, x='bmi', nbins=30, title='Distribuição de BMI')
fig.show()

In [156]:
# Distribuição da variável Gênero 
fig = px.bar(df_costs['sex'].value_counts(), title='Distribuição por Gênero')

fig.show()

- Base balanceada

In [157]:
# Distribuição da variável Fumante
fig = px.bar(df_costs['smoker'].value_counts(), title='Distribuição de Fumante')
fig.show()

- Uma variável menos balanceada, vamos cruzar com algumas variáveis

In [158]:
# Distribuição da variável Região
fig = px.bar(df_costs['region'].value_counts(), title='Distribuição por Região')
fig.show()

- Uma base balanceada

In [159]:
# BoxPlot de custos médios por idade
fig = px.box(df_costs, x='age', y='medical charges', title='BoxPlot de custos médios por idade')
fig.show()

- Percebemos que temos um aumento do custo médio conforme p aumento da idade.

- Vamos analisar com outras variáveis para fugir de vieses

In [160]:
fig = px.box(df_costs, x='sex', y='medical charges', title='BoxPlot de custos médios por Gênero')
fig.show()

- Em termos de mediana, não temos uma discrepancia de valores, mas quando obsevarmos os valores maximos vemos um gasto maior por parte do gêrenero marculino

- #### FAZER UM TESTE DE HIPOTES??

In [161]:
# BoxPlot de custos médicos por Fumante 
fig = px.box(df_costs, x='smoker', y='medical charges', title='BoxPlot de custos médios por status de fumante')
fig.show()

- Aqui conseguimos enxergar uma tendência bem clara quanto aos custos médicos em relação as pessoas que fumam e não fumam. Os fumentes realmente gastam mais.

In [162]:
# BoxPlot de custos médios por Região
fig = px.box(df_costs, x='region', y='medical charges', title='BoxPlot de custos médios por Região')
fig.show()

- Em termos de medianas, não temos uma discrepancia grande pelas regiões. Não há algo que realmente salte os olhos, a tendência de aumento não existe

In [163]:
# Plot de correlação das variáveis numéricas
corr_matrix = df_costs.select_dtypes(include=['number']).corr()


In [164]:
# Mostar Matriz de Correlação
corr_matrix

,age,bmi,children,smoker,medical charges
age,1.000000,0.109272,0.042469,-0.025019,0.299008
bmi,0.109272,1.000000,0.012759,0.003750,0.198341
children,0.042469,0.012759,1.000000,0.007673,0.067998
smoker,-0.025019,0.003750,0.007673,1.000000,0.787251
medical charges,0.299008,0.198341,0.067998,0.787251,1.000000


In [165]:
# Montar o Chart da Matriz de Correlação
fig = go.Figure()

fig.add_trace(
    go.Heatmap(
        x=corr_matrix.columns,
        y=corr_matrix.index,
        z = np.array(corr_matrix),
        text=corr_matrix.values,
        texttemplate='%{text:.2f}',
        colorscale=px.colors.diverging.RdBu,
        zmin=-1,
        zmax=1
    )
)



- Podemos constatar que há uma correlação muito forte entre a variável SMOKER e o Custo médico. A variável AGE também não fica de fora, mas devemos trabalhar com a variável mais forte quando estivermos construindo o modelo

### Preparação dos Dados

In [166]:
# Preparar os dados para o modelo
X = df_costs.drop(columns=['medical charges'])
y = df_costs['medical charges']

In [167]:
# Criar ColumnTransformer para normalizar as variáveis numéricas e OneHotEnconder para as variáveis categoricas
numeric_features = X.select_dtypes(include=['number']).columns
categorical_features = X.select_dtypes(include=['object']).columns
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features) # handle_unknown='ignore', no caso de o algoritmo considerar um dado desconhecido na hora de fazer o slip, quando não é encontrado nas duas porções.
])

In [168]:
# Dividir os dados e treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [169]:
# Aplicar ColumnTransformer nos dados de treinamento e teste
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [170]:
# Mostrar as dimeções dos conjuntos
print(f'Dados de Treinamento: {X_train.shape}')
print(f'Dados de Test: {X_test.shape}')

Dados de Treinamento: (1070, 10)
Dados de Test: (268, 10)


### Treinamento do Modelo

In [186]:
# Criar o Modelo de Bagging Regressor
bagging_model = BaggingRegressor(
    estimator=LinearRegression(),
    n_estimators=50,
    random_state=51
)

In [187]:
# Treinar o Modelo
bagging_model.fit(X_train, y_train)

BaggingRegressor(estimator=LinearRegression(), n_estimators=50, random_state=51)

### Análise dos Resultados

In [188]:
# Fazer predivisões com base no modelo treinado
y_pred = bagging_model.predict(X_test)


In [189]:
# Avaliar Métricas do Modelo
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

In [190]:
# Mostrar MAE, RMSE e R2 do Modelo
print(f'RMSE: {rmse}')
print(f'R2 Score: {r2}')
print(f'MAE: {mae}')

RMSE: 5800.35229310645
R2 Score: 0.7832891360794917
MAE: 4178.158306677919


In [181]:
# Calcular a importância das features usando os corficientes

# Obter os coeficientes de cada estimador
coef_estimator_baggingModel = np.array([estimator.coef_ for estimator in bagging_model.estimators_])
# Foi feito um list_comprehention para obter os coegicientes de cada estimador do modelo, 
# dentro de uma array no numpy, pois precisamos fazer o calculo dos coeficientes

# Calcular a Media dos coeficientes absoolutos
features_importance = np.mean(np.abs(coef_estimator_baggingModel), axis=0)
# Foi feito o calculo das medias dos coeficientes absoluto de cada coluna dent,
#  usamos o np.mean() para calcular as medias e o np.abs() para colocar os números em absoluto, 
# pegamos o objeto com os valores e indicamos que o calculo deve ser feito por coluna

# Normalizar as importâncias
features_importance = features_importance / np.sum(features_importance)

In [182]:
features_importance

array([0.2092233 , 0.11632558, 0.02772203, 0.55067103, 0.00778022,
       0.00778022, 0.02353533, 0.01607087, 0.01763473, 0.0232567 ])

In [183]:
# Obter os nomes das features
feature_names = preprocessor.get_feature_names_out()
# Queremos o nome das colunas usadas no modelo que foram criadas pelo preprocessor do algoritmo

In [184]:
# Criar um dataframe com as importâncias e os nomes
importance_df = pd.DataFrame({'features': feature_names, 'importance': features_importance})

# Ordenar o DataFrame pela valor crescente da importância
importance_df = importance_df.sort_values('importance', ascending=True)


In [185]:
# Criar um Chart para mostrar a imporntância das features
fig = px.bar(importance_df, x='importance', y='features', title='Importância das Features', orientation='h')

# Melhorando o Layout
fig.update_xaxes(tickangle=45) # previnir que o chart venha um as legendas e dimenções sem um angulo de referância 

# Mostrar chart
fig.show()


### Salvar Dados e Pre-Processador do Modelo

In [191]:
# Salvar dataframe como CSV
df_costs.to_csv('.\datasets\healthcost_cleaned.csv', index=False)

<>:2: SyntaxWarning:

invalid escape sequence '\d'

<>:2: SyntaxWarning:

invalid escape sequence '\d'

C:\Users\Carolina\AppData\Local\Temp\ipykernel_16640\3786235420.py:2: SyntaxWarning:

invalid escape sequence '\d'



In [192]:
# Salvar Preprocessor
import joblib
joblib.dump(preprocessor, '.\preprocessor_dataset_healthcosts.pkl')

<>:3: SyntaxWarning:

invalid escape sequence '\p'

<>:3: SyntaxWarning:

invalid escape sequence '\p'

C:\Users\Carolina\AppData\Local\Temp\ipykernel_16640\3677420667.py:3: SyntaxWarning:

invalid escape sequence '\p'



['.\\preprocessor_dataset_healthcosts.pkl']